<a href="https://colab.research.google.com/github/Denis2054/Context-Engineering-for-Multi-Agent-Systems/blob/main/Chapter10/Universal_Context_Engine.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Universal Context Engine

Copyright 2025-2026, Denis Rothman

**Goal:** This notebook serves as the practical runtime for the **Strategic Marketing Engine**, as architected in Chapter 9. It demonstrates how the generic, multi-domain "Glass Box" Context Engine can be seamlessly repurposed to solve a variety of real-world marketing challenges without any changes to its core code.

This notebook will:
* Connect to the Pinecone knowledge base populated by the `Data_Ingestion_Marketing.ipynb` script.
* Use the generic Control Deck templates to interact with the engine.
* Execute the marketing use cases

*Note* Make sure to run `Data_Ingestion_Marketing.ipynb` first to ingest the data for this notebook.


# I. Inititalization

## GitHub

In [1]:
print("Downloading files from public repository...")

# The -f flag tells curl to fail on an error (like 404)
!curl -Lf https://raw.githubusercontent.com/Denis2054/Context-Engineering/main/commons/engine/utils.py --output utils.py
!curl -Lf https://raw.githubusercontent.com/Denis2054/Context-Engineering/main/commons/engine/helpers.py --output helpers.py
!curl -Lf https://raw.githubusercontent.com/Denis2054/Context-Engineering/main/commons/engine/agents.py --output agents.py
!curl -Lf https://raw.githubusercontent.com/Denis2054/Context-Engineering/main/commons/engine/registry.py --output registry.py
!curl -Lf https://raw.githubusercontent.com/Denis2054/Context-Engineering/main/commons/engine/engine.py --output engine.py
# (You might want to add a check here to see if the files actually exist now)
print("✅ File download attempt finished!")

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  1823  100  1823    0     0   5193      0 --:--:-- --:--:-- --:--:--  5193
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  6479  100  6479    0     0  26342      0 --:--:-- --:--:-- --:--:-- 26444
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  9440  100  9440    0     0  22626      0 --:--:-- --:--:-- --:--:-- 22692
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  3611  100  3611    0     0   8019      0 --:--:-- --:--:-- --:--:--  8060
  % Total    % Received % Xferd  Average Speed   Tim

## Installation and client setup

In [2]:
#Installation and Client Setup

# Import the setup functions from your new utility file
import utils

# Run the installation
utils.install_dependencies()

# Initialize the OpenAI and Pinecone clients
client, pc = utils.initialize_clients()

🚀 Installing required packages...
✅ All packages installed successfully.

🔑 Initializing API clients...
   - OpenAI client initialized.
   - Pinecone client initialized.
✅ Clients initialized successfully.


## Context Engine library Import

In [3]:
# 1. Import the hardened helper functions (LLM, Embeddings, Pinecone)
import helpers

# 2. Import the specialist agent functions (Librarian, Researcher, Writer)
import agents

# 3. Import the AGENT_TOOLKIT object that knows about all the agents
from registry import AGENT_TOOLKIT

# 4. Import the main context_engine function that orchestrates the entire process
from engine import context_engine

## Render and Trace Dashboard

In [4]:
import json
import html
import markdown
from IPython.display import display, HTML

def render_trace_dashboard(trace):
    """
    Generates a high-contrast, high-readability HTML dashboard for the Context Engine Trace.
    Includes explicit token metrics and deep charcoal text for maximum clarity.
    """
    # Define CSS with hard-coded high-contrast colors and heavy font weights
    css = """
    <style>
        .dashboard-container {
            font-family: -apple-system, BlinkMacSystemFont, "Segoe UI", Roboto, Helvetica, Arial, sans-serif;
            background-color: #ffffff;
            border: 3px solid #cbd5e0;
            border-radius: 12px;
            padding: 30px;
            max-width: 100%;
            margin-top: 25px;
            color: #1a202c;
        }
        .header-section {
            border-bottom: 3px solid #2d3748;
            padding-bottom: 20px;
            margin-bottom: 25px;
            display: flex;
            justify-content: space-between;
            align-items: center;
        }
        .header-title { margin: 0; font-size: 1.8rem; color: #1a202c; font-weight: 900; }
        .header-goal { margin: 10px 0 0 0; color: #2d3748; font-size: 1.2rem; font-style: italic; font-weight: 600;}

        .status-badge {
            padding: 10px 20px;
            border-radius: 8px;
            font-weight: 900;
            font-size: 1rem;
            color: white;
            text-transform: uppercase;
        }
        .status-success { background-color: #22543d; }
        .status-failure { background-color: #742a2a; }

        .metrics-summary {
            margin-top: 10px;
            font-size: 1.2rem;
            font-weight: 900;
            color: #1a202c;
            background: #edf2f7;
            padding: 5px 12px;
            border-radius: 6px;
        }

        /* Metrics Display within Steps */
        .metrics-bar {
            display: flex;
            gap: 12px;
            margin-top: 12px;
            flex-wrap: wrap;
        }
        .metric-pill {
            background-color: #ebf4ff;
            color: #1a365d; /* High contrast dark blue */
            padding: 6px 14px;
            border-radius: 8px;
            border: 2px solid #2b6cb0;
            font-size: 0.95rem;
            font-weight: 900;
        }
        .metric-saved {
            background-color: #f0fff4;
            color: #1c4532;
            border-color: #2f855a;
        }

        /* Step Cards */
        .step-card {
            background-color: #ffffff;
            border: 2px solid #2d3748;
            border-radius: 12px;
            margin-bottom: 25px;
            overflow: hidden;
            box-shadow: 0 4px 6px rgba(0,0,0,0.05);
        }
        summary.step-header {
            padding: 20px;
            background-color: #f8fafc;
            cursor: pointer;
            list-style: none;
            display: flex;
            align-items: center;
            justify-content: space-between;
            border-bottom: 2px solid #e2e8f0;
        }
        .agent-badge {
            background-color: #1a202c;
            color: #ffffff;
            padding: 5px 14px;
            border-radius: 6px;
            font-size: 0.85rem;
            font-weight: 900;
            text-transform: uppercase;
            margin-left: 15px;
        }

        .step-content { padding: 25px; background-color: #ffffff; }

        .data-label {
            font-size: 1rem;
            text-transform: uppercase;
            color: #1a202c;
            font-weight: 900;
            margin-bottom: 12px;
            display: block;
            border-left: 4px solid #1a202c;
            padding-left: 10px;
        }

        /* THE CONTENT AREA - High Visibility for Headings */
        .rendered-content {
            background-color: #ffffff;
            border: 2px solid #e2e8f0;
            border-left: 8px solid #2b6cb0;
            padding: 20px;
            color: #1a202c !important;
            line-height: 1.7;
            font-size: 1.1rem;
            font-weight: 500;
        }
        .rendered-content h1, .rendered-content h2, .rendered-content h3,
        .rendered-content h4, .rendered-content h5, .rendered-content h6 {
            color: #1a202c !important;
            font-weight: 900 !important;
            margin-top: 1.5em;
            margin-bottom: 0.5em;
        }
        .rendered-content strong, .rendered-content b {
            font-weight: 900;
            color: #000000;
        }

        .json-box {
            background-color: #1a202c;
            color: #f7fafc;
            padding: 20px;
            border-radius: 10px;
            font-family: "SFMono-Regular", Consolas, monospace;
            font-size: 0.95rem;
            overflow-x: auto;
        }

        .final-output-card {
            border: 5px solid #22543d;
            background-color: #f0fff4;
            border-radius: 12px;
            padding: 30px;
            margin-top: 50px;
            color: #1a202c;
        }
    </style>
    """

    status_class = "status-success" if trace.status == "Success" else "status-failure"

    dashboard_html = f"""
    {css}
    <div class="dashboard-container">
        <div class="header-section">
            <div>
                <h1 class="header-title">Context Engine Trace</h1>
                <p class="header-goal">"{html.escape(trace.goal)}"</p>
            </div>
            <div style="text-align: right;">
                <span class="status-badge {status_class}">{trace.status}</span>
                <div class="metrics-summary">
                    ⏱️ TIME: {trace.duration:.2f}s
                </div>
            </div>
        </div>

        <div class="steps-container">
            <h2 style="color: #1a202c; margin-bottom: 25px; font-size: 1.4rem; font-weight: 900; text-transform: uppercase;">Execution Workflow</h2>
    """

    for step in trace.steps:
        # Prepare inputs and outputs
        try:
            resolved_ctx = json.dumps(step['resolved_context'], indent=2)
        except:
            resolved_ctx = str(step.get('resolved_context', 'N/A'))

        output_raw = step.get('output', 'N/A')
        rendered_html = ""

        def render_md(text):
            return markdown.markdown(text) if text else "No content recorded."

        if isinstance(output_raw, dict):
            content_found = False
            for key in ['summary', 'answer_with_sources', 'answer', 'output', 'content']:
                if key in output_raw and isinstance(output_raw[key], str):
                    rendered_html = f'<div class="rendered-content">{render_md(output_raw[key])}</div>'
                    content_found = True
                    break
            if not content_found:
                rendered_html = f'<div class="json-box">{html.escape(json.dumps(output_raw, indent=2))}</div>'
        else:
            rendered_html = f'<div class="rendered-content">{render_md(str(output_raw))}</div>'

        # Metrics Extraction
        t_in = step.get('tokens_in', '??')
        t_out = step.get('tokens_out', '??')
        t_saved = step.get('tokens_saved', 0)

        metrics_html = f"""
        <div class="metrics-bar">
            <span class="metric-pill">📥 IN: {t_in}</span>
            <span class="metric-pill">📤 OUT: {t_out}</span>
        """
        if t_saved > 0:
            metrics_html += f'<span class="metric-pill metric-saved">📉 SAVED: {t_saved}</span>'
        metrics_html += "</div>"

        # Build Step Card
        step_html = f"""
            <details class="step-card" open>
                <summary class="step-header">
                    <div style="display:flex; flex-direction:column; align-items:flex-start;">
                        <div>
                            <span style="font-weight:900; font-size: 1.3rem; color: #1a202c;">STEP {step['step']}</span>
                            <span class="agent-badge">{step['agent']}</span>
                        </div>
                        {metrics_html}
                    </div>
                    <span style="font-weight: 900; color: #ffffff; background: #1a202c; padding: 6px 14px; border-radius: 6px; font-size: 0.8rem;">OPEN LOGS</span>
                </summary>
                <div class="step-content">
                    <div style="margin-bottom:30px;">
                        <span class="data-label">Input Context (State)</span>
                        <details><summary style="font-size:0.9rem; font-weight:900; color: #2b6cb0; cursor:pointer; margin-bottom:10px;">▶ View Resolved Source Data</summary>
                        <div class="json-box">{html.escape(resolved_ctx)}</div></details>
                    </div>
                    <div>
                        <span class="data-label">Agent Output</span>
                        {rendered_html}
                    </div>
                </div>
            </details>
        """
        dashboard_html += step_html

    # Final Result Section
    if trace.final_output:
        final_content = trace.final_output
        if isinstance(final_content, dict):
            final_content = final_content.get('summary', final_content.get('content', str(final_content)))

        dashboard_html += f"""
        <div class="final-output-card">
            <div style="color:#1c4532; font-size: 1.5rem; font-weight:1000; margin-bottom:20px; text-transform: uppercase; letter-spacing: 2px; border-bottom: 3px solid #22543d; padding-bottom: 12px;">Final Orchestration Result</div>
            <div style="font-size: 1.25rem; font-weight: 700; line-height: 1.8;">{markdown.markdown(str(final_content))}</div>
        </div>
        """

    dashboard_html += "</div>"
    display(HTML(dashboard_html))

## Engine Room

In [5]:
# === ENGINE ROOM: The Main Execution Function ===
# This function contains all the logic to run the engine.
# We define it here so our final cell can be very simple.

import logging
import pprint
import json  # <--- Added for type handling
from IPython.display import display, Markdown

# === ENGINE ROOM: The Main Execution Function (Visualizer Update) ===

import logging
import pprint
import json
from IPython.display import display, Markdown

def execute_and_display(goal, config, client, pc, moderation_active):
    """
    Runs the context engine with HTML dashboard visualization.
    """
    # --- PRE-FLIGHT MODERATION CHECK (on user input) ---\n
    if moderation_active:
        print("--- [Safety Guardrail] Performing Pre-Flight Moderation Check on Goal ---")
        moderation_report = helpers.helper_moderate_content(text_to_moderate=goal, client=client)

        if moderation_report["flagged"]:
            print("\n🛑 Goal failed pre-flight moderation. Execution halted.")
            pprint.pprint(moderation_report)
            return

    logging.info(f"******** Starting Engine for Goal: '{goal}' **********\\n")

    # 1. Run the Context Engine
    result, trace = context_engine(
        goal,
        client=client,
        pc=pc,
        **config
    )

    # --- POST-FLIGHT MODERATION CHECK (on AI output) ---\n
    if result and moderation_active:
        # Flatten result for checking
        text_to_check = str(result)
        if isinstance(result, (dict, list)):
            text_to_check = json.dumps(result)

        moderation_report = helpers.helper_moderate_content(text_to_moderate=text_to_check, client=client)

        if moderation_report["flagged"]:
            print("\n🛑 Generated output failed post-flight moderation and will be redacted.")
            result = "[Content flagged as potentially harmful by moderation policy and has been redacted.]"
            # We also scrub the trace result to prevent display of harmful content
            trace.final_output = result

    # 2. Render the HTML Dashboard
    # This replaces the raw text printouts with the visualizer
    if trace:
        render_trace_dashboard(trace)
    else:
        print("Engine failed to initialize trace.")

In (Cell 8), the query was simple, so the LLM found the structure correctly. In the failing runs (Cell 11, 12), the queries were more complex ("Analyze...", "Write a pitch..."), causing the LLM to hallucinate a slightly different JSON format. The patch above forces the LLM to adhere to the required structure every time.

In [6]:
# =============================================================================
# 🛠️ HOTFIX: ROBUST PLANNER PATCH
# =============================================================================
# Run this cell to overwrite the default planner with a strict-schema version.
# This fixes the "NoneType object has no attribute 'get'" error.

import engine
import json
import logging
from helpers import call_llm_robust

def planner_robust_patch(goal, capabilities, client, generation_model):
    """
    A patched version of the Planner that enforces strict JSON schema for inputs.
    """
    logging.info("[Planner-Patch] Activated. Generating strict schema plan...")

    # UPGRADE: Explicitly showing the required schema in the prompt
    system_prompt = f"""
You are the strategic core of the Context Engine. Analyze the user's high-level GOAL and create a step-by-step EXECUTION PLAN.

AVAILABLE CAPABILITIES
---
{capabilities}
---
END CAPABILITIES

INSTRUCTIONS:
1. The output MUST be a single JSON object with a "plan" key containing a list of step objects.
2. CRITICAL: Every step object MUST strictly follow this schema:
   {{
      "step": <integer>,
      "agent": "<Agent Name>",
      "input": {{
          "<input_key>": "<input_value>"
      }}
   }}
   (You MUST wrap the agent arguments inside the "input" object. Do not place them at the root of the step object.)

3. Use Context Chaining: format "$$STEP_N_OUTPUT$$" for values requiring previous outputs.
"""
    try:
        plan_json_string = call_llm_robust(
            system_prompt,
            goal,
            client=client,
            generation_model=generation_model,
            json_mode=True
        )
        plan_data = json.loads(plan_json_string)

        # Extra safety: Check if 'plan' key exists, if not, assume the list is the plan
        if "plan" not in plan_data and isinstance(plan_data, list):
             return plan_data
        return plan_data["plan"]

    except Exception as e:
        logging.error(f"Planner failed to generate a valid plan. Error: {e}")
        raise e

# Apply the patch to the imported engine module
engine.planner = planner_robust_patch
print("✅ Engine successfully patched with Robust Planner.")

✅ Engine successfully patched with Robust Planner.


## Control Deck configuration

In [7]:
# 1. Define all configuration variables for this run in a dictionary
config = {
    "index_name": 'genai-mas-mcp-ch3',
    "generation_model": "gpt-5.1",
    "embedding_model": "text-embedding-3-small",
    "namespace_context": 'ContextLibrary',
    "namespace_knowledge": 'KnowledgeStore'
}

#III.CONTROL DECKS

=== CONTROL DECK: Define Goal and Run Engine ===
This is the main interactive cell.
1. Change the 'goal' variable to your desired task.
2. Run this cell.


## I Marketing

In [8]:
#@title CONTROL DECK: Moderation
# 1. Define a simple, safe goal to test the moderation workflow.
goal = "Summarize the key points of the QuantumDrive"

# 2. Define the standard configuration.
config = {
    "index_name": 'genai-mas-mcp-ch3',
    "generation_model": "gpt-5.1",
    "embedding_model": "text-embedding-3-small",
    "namespace_context": 'ContextLibrary',
    "namespace_knowledge": 'KnowledgeStore'
}

# 3. Call the execution function with moderation explicitly activated.
execute_and_display(goal, config, client, pc, moderation_active=True)

--- [Safety Guardrail] Performing Pre-Flight Moderation Check on Goal ---


In [9]:
#@title Product Marketing Copy Generation
goal = "Analyze the ChronoTech press release and summarize their core product messaging and value proposition. Please cite your sources."


# 2. Use the standard configuration
config = {
    "index_name": 'genai-mas-mcp-ch3',
    "generation_model": "gpt-5.1", # or your preferred model
    "embedding_model": "text-embedding-3-small",
    "namespace_context": 'ContextLibrary',
    "namespace_knowledge": 'KnowledgeStore'
}

# 3. Call the execution function
execute_and_display(goal, config, client, pc,moderation_active=False)

In [11]:
#@title Writing a brand pitch recommendation

# 1. Define the Goal: A creative or factual task that is deliberately
#    outside the scope of the documents in the knowledge base.
#    - DOMAIN: Universal test applicable to any curated knowledge base.
#    - KEY CAPABILITY: Tests the `Researcher` agent's ability to report a
#      negative finding and the `Writer` agent's ability to handle it gracefully,
#      preventing hallucination.
# goal = "[INSERT YOUR OUT-OF-SCOPE GOAL HERE]"


# =CONTROL DECK 3: A persuasive pitch ===
goal = "Write a persuasive pitch on our brand tone and voice guide"


# 2. Use the same configuration dictionary
config = {
    "index_name": 'genai-mas-mcp-ch3',
    "generation_model": "gpt-5.1", # or your preferred model
    "embedding_model": "text-embedding-3-small",
    "namespace_context": 'ContextLibrary',
    "namespace_knowledge": 'KnowledgeStore'
}

# 3. Call the execution function
execute_and_display(goal, config, client, pc,moderation_active=False)

## II Legal

In [12]:
#@title CONTROL DECK: Moderation
# 1. Define a simple, safe goal to test the moderation workflow.

# November 20, 2025 goal update:
# The former goal contained the the term "summarize" creating ambiguity.
#goal = "Summarize the key points of the Non-Disclosure Agreement."

# The new goal now explains that the task is not to directly summarize but first retrieve data thus clarifying the ambiguity
goal = "First, retrieve the content of the Non-Disclosure Agreement (NDA) from the knowledge base. Then, summarize its key points."
# The system will then function correctly, sanitize the information and continue the process

# 2. Define the standard configuration.
config = {
    "index_name": 'genai-mas-mcp-ch3',
    "generation_model": "gpt-5.1",
    "embedding_model": "text-embedding-3-small",
    "namespace_context": 'ContextLibrary',
    "namespace_knowledge": 'KnowledgeStore'
}

# 3. Call the execution function with moderation explicitly activated.
execute_and_display(goal, config, client, pc, moderation_active=True)

--- [Safety Guardrail] Performing Pre-Flight Moderation Check on Goal ---


In [13]:
#@title CONTROL DECK TEMPLATE 1: High-Fidelity RAG

# 1. Define the Goal: A research query that requires a verifiable, cited answer.
#    - DOMAIN: Any knowledge-intensive field (e.g., legal, medical, financial).
#    - KEY CAPABILITY: Tests the high-fidelity `Researcher` agent and its ability
#      to retrieve text with `source` metadata and generate citations.
# goal = "[INSERT YOUR HIGH-FIDELITY RESEARCH GOAL HERE]"

# === CONTROL DECK 1: High-Fidelity RAG in a Legal Context ===
goal = "What are the key confidentiality obligations in the Service Agreement v1, and what is the termination notice period? Please cite your sources."

# === CONTROL DECK 1 (LIMIT TEST): Sanitization of Legal Testimony ===
#goal = "What did Mr. Smith advise his client regarding the assets?"

# 2. Use the standard configuration
config = {
    "index_name": 'genai-mas-mcp-ch3',
    "generation_model": "gpt-5.1", # or your preferred model
    "embedding_model": "text-embedding-3-small",
    "namespace_context": 'ContextLibrary',
    "namespace_knowledge": 'KnowledgeStore'
}

# 3. Call the execution function
execute_and_display(goal, config, client, pc,moderation_active=False)